In [29]:
import json
import torch
from torch.utils.data import Dataset
import os
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import torch.optim as optim

In [30]:
class WSIEmbeddingsDataset(Dataset):
    def __init__(self, base_dir="cases"):
        self.metadata = []

        case_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

        if "GENERAL_METADATA" in case_dirs:
            case_dirs.remove("GENERAL_METADATA")

        for case_id in case_dirs:
            case_dir = os.path.join(base_dir, case_id)
            metadata_path = os.path.join(case_dir, "aggregated_data", f'{case_id}_data.json')

            if not os.path.exists(metadata_path):
                print(f"No metadata found for case {case_id}, skipping...")
                continue
            
            with open(metadata_path, 'r') as f:
                case_metadata = json.load(f)

            for sample in case_metadata["biospecimen"]["biospecimen_data"]:
                if sample["sample_type"] != "Primary Tumor":
                    continue

                for slide in sample["slides"]:
                    censored = 0
                    slide_barcode = slide["slide_barcode"]
                    survival_time = case_metadata["clinical"]["clinical_patient_data"].get("days_to_death")
                    
                    if survival_time is None:
                        survival_time = case_metadata["clinical"]["clinical_patient_data"].get("days_to_last_followup")
                        censored = 1

                    if survival_time is None:
                        continue

                    self.metadata.append({
                        "json_file": metadata_path,
                        "case_id": case_id,
                        "slide_barcode": slide_barcode,
                        "survival_time": survival_time,
                        "censored": censored
                    })
    
    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, index):
        item = self.metadata[index]
        
        with open(item["json_file"], 'r') as f:
            case_metadata = json.load(f)

        slide_data = None
        for sample in case_metadata["biospecimen"]["biospecimen_data"]:
            if sample["sample_type"] == "Primary Tumor":
                for slide in sample["slides"]:
                    if slide["slide_barcode"] == item["slide_barcode"]:
                        slide_data = slide
                        break
            if slide_data:
                break
        
        if not slide_data:
            raise ValueError(f"Slide data not found for {item['slide_barcode']} in {item['json_file']}.")

        embedding = slide_data["embedding"]
        survival_time = case_metadata["clinical"]["clinical_patient_data"]["days_to_death"]
        censored = 0

        if survival_time is None:
            survival_time = case_metadata["clinical"]["clinical_patient_data"]["days_to_last_followup"]
            censored = 1

        embedding_tensor = torch.tensor(embedding, dtype=torch.float32)
        survival_time_tensor = torch.tensor(int(survival_time), dtype=torch.float32)
        censored_tensor = torch.tensor(censored, dtype=torch.float32)

        return embedding_tensor, survival_time_tensor, censored_tensor

In [31]:
class SurvivalNN(nn.Module):
    def __init__(self, embedding_size):
        super(SurvivalNN, self).__init__()
        
        # Define a simple feedforward neural network
        self.fc1 = nn.Linear(embedding_size + 2, 64)  # +2 for survival time and censored
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Output single value for survival prediction

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [32]:
dataset = WSIEmbeddingsDataset(base_dir="cases_TEST")
all_data = DataLoader(dataset)

for data in all_data:
    print(f"Data: {data}")

train_size = int(0.75 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset)
test_loader = DataLoader(test_dataset)

torch.manual_seed(42)

embedding_size = 2048
model = SurvivalNN(embedding_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    
    for batch in train_loader:
        embedding, survival_time, censored = batch
        
        input_data = torch.cat((embedding, survival_time.unsqueeze(1), censored.unsqueeze(1)), dim=1)
        
        output = model(input_data)
        loss = criterion(output, survival_time.unsqueeze(1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
    

Data: [tensor([[0.0057, 0.1384, 0.0133,  ..., 0.0055, 0.0206, 0.0521]]), tensor([28.]), tensor([1.])]
Data: [tensor([[0.0031, 0.1424, 0.0095,  ..., 0.0036, 0.0367, 0.0660]]), tensor([28.]), tensor([1.])]
Data: [tensor([[0.0103, 0.0929, 0.0059,  ..., 0.0031, 0.0210, 0.0825]]), tensor([0.]), tensor([1.])]
Data: [tensor([[0.0029, 0.1262, 0.0075,  ..., 0.0022, 0.0265, 0.1581]]), tensor([0.]), tensor([1.])]
Epoch [1/10], Loss: 785.9708
Epoch [2/10], Loss: 779.6328
Epoch [3/10], Loss: 775.1263
Epoch [4/10], Loss: 784.5565
Epoch [5/10], Loss: 775.0576
Epoch [6/10], Loss: 757.9441
Epoch [7/10], Loss: 748.8263
Epoch [8/10], Loss: 734.5081
Epoch [9/10], Loss: 691.5579
Epoch [10/10], Loss: 732.8704
